In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Sequential

# Read time series info

In [2]:
m4_info = pd.read_csv("../archive/m4_info.csv")
m4_info = m4_info.set_index("M4id").T.to_dict("dict")
m4_info["Y1"]

{'category': 'Macro',
 'Frequency': 1,
 'Horizon': 6,
 'SP': 'Yearly',
 'StartingDate': '01-01-79 12:00'}

# Read time series datasets

In [3]:
datasets = {}
types = ["Daily", "Hourly", "Monthly", "Quarterly", "Weekly", "Yearly"]

In [4]:
for dataset in ["train", "test"]:
    datasets[dataset] = {}
    for series_type in tqdm(types):
        df = pd.read_csv(f"../archive/{series_type}-{dataset}.csv")
        df.insert(0, series_type, df["V1"])
        df = df.shift(-1, axis=1)
        df = df.set_index(series_type).T.to_dict("list")
        df = {k: np.array(v)[:-1] for k, v in df.items()}
        df = {k: v[: np.where(~np.isnan(v))[0][-1] + 1] for k, v in df.items()}
        datasets[dataset][series_type] = df

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:02<00:00,  2.01it/s]


In [5]:
datasets["train"]["Daily"]["D1"]

array([1017.1, 1019.3, 1017. , ..., 2022.1, 2031.6, 2029.7])

# Metrics

In [5]:
def smape(true_values, predictions):
    return 200 * np.mean(
        np.abs(predictions - true_values) /
        (np.abs(predictions) + np.abs(true_values))
    )


def mase(train_values, true_values, predictions, frequency):
    denom = np.mean(
        np.abs(train_values[-frequency:] -
               train_values[-2 * frequency: -frequency])
    )

    return 0 if denom == 0 else np.mean(np.abs(predictions - true_values)) / denom

In [7]:
def naive2_adjust(series, seasonality_period):
    decomposition = seasonal_decompose(
        series,
        model="multiplicative",
        period=seasonality_period,
        extrapolate_trend="freq",
    )

    autocorrelation = sm.tsa.acf(series, nlags=seasonality_period)

    if np.max(np.abs(autocorrelation[1:])) > 0.9:
        adjusted_series = series / decomposition.seasonal
        seasonal_component = decomposition.seasonal[-seasonality_period:]
    else:
        adjusted_series = series
        seasonal_component = np.ones(seasonality_period)

    return adjusted_series, seasonal_component

# Naive S

In [8]:
smape_scores = []
mase_scores = []

for series_type in types:
    for series, values in datasets["train"][series_type].items():

        frequency = m4_info[series]["Frequency"]
        horizon = m4_info[series]["Horizon"]

        true_values = datasets["test"][series_type][series]
        prediction = np.tile(values[-horizon:], horizon // frequency + 1)[
            :horizon
        ]  # naive S

        smape_scores.append(smape(true_values, prediction))
        mase_scores.append(mase(values, true_values, prediction, frequency))

np.mean(smape_scores), np.mean(mase_scores)

(np.float64(18.746165312454934), np.float64(5.845256207129324))

# Theta

In [9]:
def theta_forecast(series, horizon, seasonality_period):
    adjusted_series, seasonal_component = naive2_adjust(series, seasonality_period)

    n = len(adjusted_series)
    time = np.arange(1, n + 1)

    X = sm.add_constant(time)
    model_lr = sm.OLS(adjusted_series, X).fit()
    trend_forecast = model_lr.predict(
        sm.add_constant(np.arange(n + 1, n + horizon + 1))
    )

    ses_model = SimpleExpSmoothing(adjusted_series).fit()
    ses_forecast = ses_model.forecast(horizon)

    combined_forecast = (trend_forecast + ses_forecast) / 2

    if len(seasonal_component) > 0:
        seasonal_component_repeated = np.tile(
            seasonal_component, horizon // len(seasonal_component) + 1
        )[:horizon]
        final_forecast = combined_forecast * seasonal_component_repeated
    else:
        final_forecast = combined_forecast

    return final_forecast

In [10]:
smape_scores = []
mase_scores = []

for series_type in types:
    for series, values in datasets["train"][series_type].items():

        frequency = m4_info[series]["Frequency"]
        horizon = m4_info[series]["Horizon"]

        true_values = datasets["test"][series_type][series]
        prediction = theta_forecast(values, horizon, frequency)

        smape_scores.append(smape(true_values, prediction))
        mase_scores.append(mase(values, true_values, prediction, frequency))

np.mean(smape_scores), np.mean(mase_scores)

/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retval

(np.float64(15.836007936416541), np.float64(7.498211596201757))

# Holt

In [11]:
def holt_forecast(series, horizon, seasonality_period):
    adjusted_series, seasonal_component = naive2_adjust(
        series, seasonality_period)

    holt_model = ExponentialSmoothing(
        adjusted_series, trend="add", seasonal=None).fit()
    holt_forecast = holt_model.forecast(horizon)

    if len(seasonal_component) > 0:
        seasonal_component_repeated = np.tile(
            seasonal_component, horizon // len(seasonal_component) + 1
        )[:horizon]
        final_forecast = holt_forecast * seasonal_component_repeated
    else:
        final_forecast = holt_forecast

    return final_forecast

In [12]:
smape_scores = []
mase_scores = []

for series_type in types:
    for series, values in datasets["train"][series_type].items():

        frequency = m4_info[series]["Frequency"]
        horizon = m4_info[series]["Horizon"]

        true_values = datasets["test"][series_type][series]
        prediction = holt_forecast(values, horizon, frequency)

        smape_scores.append(smape(true_values, prediction))
        mase_scores.append(mase(values, true_values, prediction, frequency))

np.mean(smape_scores), np.mean(mase_scores)

/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/dimronto/anaconda3/envs/TimeSeriesM4/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retval

(np.float64(14.374776480859769), np.float64(4.145163831652946))

# Comb

# RNN

In [26]:
def preprocess(series, seasonality_period):
    n = len(series)
    time = np.arange(n).reshape(-1, 1)

    model = LinearRegression().fit(time, series)
    trend = model.predict(time)
    detrended_series = series - trend

    autocorrelation = sm.tsa.acf(series, nlags=seasonality_period)

    if np.max(np.abs(autocorrelation[1:])) > 0.9:
        decomposition = seasonal_decompose(
            series,
            model="multiplicative",
            period=seasonality_period,
            extrapolate_trend="freq",
        )
        seasonal_component = decomposition.seasonal
        deseasonalized_series = detrended_series / seasonal_component
    else:
        deseasonalized_series = detrended_series
        seasonal_component = np.ones(len(series))

    return deseasonalized_series, seasonal_component, trend

In [7]:
def prepare_data(series, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i: i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)

In [20]:
def build_rnn(input_shape):
    model = Sequential(
        [
            Input(shape=input_shape),
            LSTM(64, activation="relu", return_sequences=True),
            LSTM(32, activation="relu"),
            Dense(1),
        ]
    )

    model.compile(optimizer="adam", loss="mse")
    return model

In [16]:
def rnn_forecast(series, horizon, seasonality_period):
    deseasonalized_series, seasonal_component, trend = preprocess(
        series, seasonality_period
    )

    window_size = seasonality_period
    X, y = prepare_data(deseasonalized_series, window_size)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    rnn_model = build_rnn(input_shape=(window_size, 1))
    rnn_model.fit(X, y, epochs=20, batch_size=16, verbose=0)

    prediction_input = deseasonalized_series[-window_size:].reshape(
        (1, window_size, 1))
    predictions = []

    for _ in range(horizon):
        next_pred = rnn_model.predict(prediction_input, verbose=0)[0, 0]
        predictions.append(next_pred)

        next_pred = np.array(next_pred).reshape((1, 1, 1))
        prediction_input = np.append(
            prediction_input[:, 1:], next_pred, axis=1)

    future_trend = trend[-1] + \
        np.arange(1, horizon + 1) * (trend[-1] - trend[-2])
    seasonal_pattern = np.tile(
        seasonal_component[-seasonality_period:], horizon // seasonality_period + 1
    )[:horizon]

    final_forecast = (np.array(predictions) * seasonal_pattern) + future_trend

    return final_forecast

In [27]:
smape_scores = []
mase_scores = []

for series_type in types:
    progress_bar = tqdm(
        datasets["train"][series_type].items(), desc="Processing Series", leave=True
    )

    for series, values in progress_bar:

        frequency = m4_info[series]["Frequency"]
        horizon = m4_info[series]["Horizon"]

        true_values = datasets["test"][series_type][series]
        prediction = rnn_forecast(values, horizon, frequency)

        smape_scores.append(smape(true_values, prediction))
        mase_scores.append(mase(values, true_values, prediction, frequency))

        progress_bar.set_postfix(
            {"SMAPE Mean": np.mean(smape_scores),
             "MASE Mean": np.mean(mase_scores)}
        )

np.mean(smape_scores), np.mean(mase_scores)

Processing Series:   9%|▉         | 391/4227 [21:37<2:49:09,  2.65s/it, SMAPE Mean=3.45, MASE Mean=14.6] 

: 

: 